In [1]:
pip install langchain langchain-openai pydantic

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
import json
from datetime import date

# 1. Define structured output
class Task(BaseModel):
    task_name: str = Field(description="Name of the task.")
    estimated_days: int = Field(description="Estimated days to complete the task.")
    team_member: str = Field(description="Assigned team member's name.")

class ProjectPlan(BaseModel):
    project_description: str = Field(description="Overall description of the project.")
    tasks: list[Task] = Field(description="List of detailed tasks for the project.")
    risks: list[str] = Field(description="Potential risks associated with the project plan.")
    steps: list[str] = Field(description="Steps to Write Better AI Prompts for Project Manager.")
    phase_details: dict = Field(description="Specific details for this project phase.")
    start_date: date = Field(description="The official start date of the project.")



# 2. Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# 3. Define parser
parser = PydanticOutputParser(pydantic_object=ProjectPlan)

# 4. Define prompts for each phase
prompts = {
    "Phase1": """You are an experienced project manager.
    You need to develop a project charter for leadership approval.
    Include:
        "Project objectives"
        "Scope and boundaries"
        "Stakeholders and roles"
        "Timeline estimate"
        "Risk overview"
    Format the output as a structured outline or table.
    Important: All dates must be in the year 2026 and  it should be equal or greater current date.
    Exclude post-launch analytics.
    Format as a timeline with linked tasks and sections for owners, status, and deadlines
    """,

    "Phase2": """You are a PMP-certified project manager .
    You need to create a full project plan that includes:
        "Task breakdown"
        "Assigned roles"
        "Timelines and dependencies"
        "Resource allocation"
        "Communication plan"
    Exclude post-launch analytics.
    Important: All dates must be in the year 2026 and  it should be equal or greater current date.
    Format as a timeline with linked tasks and sections for owners, status, and deadlines.""",

    "Phase3": """You are a project team lead at a software company midway through a sprint.
    You need to summarize progress and assign next steps based on the latest team sync.
    Include:
        "Completed tasks"
        "Remaining tasks and blockers"
        "Assigned owners"
        "Estimated time to completion"
    Format as a status update report in bullet points or a task list.
    Important: All dates must be in the year 2026 and  it should be equal or greater current date.
    """,

    "Phase4": """You are a.
    You need to create a project performance dashboard summary for stakeholders.
    Include:
        "Budget vs. actual"
        "Schedule variance"
        "Key risk flags"
        "Change requests submitted"
        "Recommendations for course correction"
    Important: All dates must be in the year 2026 and  it should be equal or greater current date.
    Format the output as a brief executive summary followed by a bullet-point list.""",

    "Phase5": """You are a project manager.
    You need to write a project closure report for internal review.
    Include:
        "Final outcomes vs. goals"
        "Lessons learned"
        "Budget summary"
        "Recommendations for future campaigns"
    Do not include raw campaign performance data (assume it’s attached).
    Important: All dates must be in the year 2026 and  it should be equal or greater current date.
    Format as a narrative summary with section headers."""
}

# 5. Run chain for each phase
arrphase = ("Phase1", "Phase2", "Phase3", "Phase4", "Phase5")
#project_goal_input = input("Enter a subject: ")
project_goal_input = "Develop a basic e-commerce website using Python and Django."
project_start_date = date(2026, 2, 6)  # YYYY, MM, DD


for phase in arrphase:
    print(f"\n--- Running {phase} Audit ---")
    prompt = ChatPromptTemplate.from_messages([
        ("system", prompts[phase]),
        ("human", "Generate the structured project plan in JSON format. {format_instructions}")
    ]).partial(format_instructions=parser.get_format_instructions())

    planning_chain = prompt | llm | parser
    response = planning_chain.invoke({"project_goal":project_goal_input,"start_date": str(project_start_date)})

    #print(response)

    print("--- Generated Project Plan ---")
    print(f"Project Description: {response.project_description}\n")
    
    print("Tasks:")
    for i, task in enumerate(response.tasks):
        print(f"* {i+1}. {task.task_name} ({task.estimated_days} days) - Assigned to: {task.team_member}")
    
    print("\nRisks:")
    for i, risk in enumerate(response.risks):
        print(f"* {i+1}. {risk}")
    
    print("\nSteps for Project Manager:")
    for i, step in enumerate(response.steps):
        print(f"* {i+1}. {step}")
        
    print("Project Phase:")
    for i, phasedetail in enumerate(response.phase_details):
        value = response.phase_details[phasedetail]
         
        if isinstance(value, str):
            print(f"* {i+1}. {phasedetail} -  {value}")
        elif isinstance(value, dict):
            print(f"* {i+1}. {phasedetail}: ")
            for k, v in value.items():
                print(f"   - {k}: {v}")



--- Running Phase1 Audit ---
--- Generated Project Plan ---
Project Description: Development of a new customer relationship management system to improve customer interactions and increase sales.

Tasks:
* 1. Gather requirements from stakeholders (5 days) - Assigned to: John Doe
* 2. Design system architecture (10 days) - Assigned to: Jane Smith
* 3. Develop core features (20 days) - Assigned to: Alex Johnson
* 4. Test and QA (10 days) - Assigned to: Emily Brown
* 5. Deploy system (5 days) - Assigned to: Chris Lee

Risks:
* 1. Lack of stakeholder involvement
* 2. Technical challenges during development
* 3. Budget constraints

Steps for Project Manager:
* 1. Define project scope and objectives
* 2. Identify key stakeholders and their roles
* 3. Create a project plan including timeline and milestones
* 4. Allocate resources and assign tasks to team members
* 5. Monitor progress and address any issues that arise
Project Phase:
* 1. scope_and_boundaries - <class 'str'> - The system will i